In [20]:
import os
import pandas as pd
import numpy as np
import json
import random, string
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
maindir = "D:/Blood/data" # Directory with files
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

In [31]:
# Define target columns
target_columns = ['hdl_cholesterol_human', 'hemoglobin(hgb)_human', 'cholesterol_ldl_human']

# Separate features and targets
X_train = train.drop(columns=target_columns)
y_train = train[target_columns]

X_test = test

In [32]:
from sklearn.preprocessing import LabelEncoder

encoders = {}
for col in target_columns:
    encoder = LabelEncoder()
    y_train[col] = encoder.fit_transform(y_train[col])
    encoders[col] = encoder

C:\Users\User\AppData\Local\Temp\ipykernel_61784\2784250006.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train[col] = encoder.fit_transform(y_train[col])
C:\Users\User\AppData\Local\Temp\ipykernel_61784\2784250006.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train[col] = encoder.fit_transform(y_train[col])
C:\Users\User\AppData\Local\Temp\ipykernel_61784\2784250006.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [40]:
from sklearn.metrics import mutual_info_score

# Calculate MI for each pair of targets
for i in range(len(target_columns)):
    for j in range(i + 1, len(target_columns)):
        mi = mutual_info_score(y_train[target_columns[i]], y_train[target_columns[j]])
        print(f"Mutual Information between {target_columns[i]} and {target_columns[j]}: {mi:.2f}")


Mutual Information between hdl_cholesterol_human and hemoglobin(hgb)_human: 0.01
Mutual Information between hdl_cholesterol_human and cholesterol_ldl_human: 0.01
Mutual Information between hemoglobin(hgb)_human and cholesterol_ldl_human: 0.01


In [50]:
# Load selected features from JSON
with open("selected_features.json", "r") as file:
    selected_features = json.load(file)

In [52]:
# Apply selected features to train and test data
X_train_target1 = X_train[selected_features["Target1"]]
X_train_target2 = X_train[selected_features["Target2"]]
X_train_target3 = X_train[selected_features["Target3"]]

X_test_target1 = X_test[selected_features["Target1"]]
X_test_target2 = X_test[selected_features["Target2"]]
X_test_target3 = X_test[selected_features["Target3"]]

In [64]:
from sklearn.model_selection import StratifiedKFold

# Map targets to their respective datasets
target_to_dataset = {
    "hdl_cholesterol_human": X_train_target1,
    "hemoglobin(hgb)_human": X_train_target2,
    "cholesterol_ldl_human": X_train_target3,
}

# Redo Stratified K-Fold for each target
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Dictionary to store splits for each target
splits = {}

# Process each target
for target, X_target in target_to_dataset.items():
    
    # Get the corresponding labels
    y_target = y_train[target]
    
    # Compute Stratified K-Fold splits
    splits[target] = list(skf.split(X_target, y_target))

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

baseline_model = RandomForestClassifier(random_state=42)

results = {}

for target, fold_splits in splits.items():
    print(f"\n=== Training Models for Target: {target} ===")

    # Access target-specific dataset and labels
    X_target = target_to_dataset[target]
    y_target = y_train[target]

    results[target] = []

    # Iterate through folds
    for fold, (train_idx, val_idx) in enumerate(fold_splits):
        # Split the data
        X_train_fold = X_target.iloc[train_idx]
        X_val_fold = X_target.iloc[val_idx]
        y_train_fold = y_target.iloc[train_idx]
        y_val_fold = y_target.iloc[val_idx]

        # Train the model
        model = baseline_model
        model.fit(X_train_fold, y_train_fold)

        # Validate the model
        y_val_pred = model.predict(X_val_fold)
        accuracy = accuracy_score(y_val_fold, y_val_pred)
        print(f"  Fold {fold + 1}: Accuracy = {accuracy:.4f}")

        # Store fold results
        results[target].append(accuracy)

# Summary of results
for target, accuracies in results.items():
    print(f"\n{target} Average Accuracy: {sum(accuracies) / len(accuracies):.4f}")



=== Training Models for Target: hdl_cholesterol_human ===
  Fold 1: Accuracy = 0.6747
  Fold 2: Accuracy = 0.6899
  Fold 3: Accuracy = 0.6800
  Fold 4: Accuracy = 0.6918
  Fold 5: Accuracy = 0.6743

=== Training Models for Target: hemoglobin(hgb)_human ===
  Fold 1: Accuracy = 0.9083
  Fold 2: Accuracy = 0.9110
  Fold 3: Accuracy = 0.9091
  Fold 4: Accuracy = 0.9091
  Fold 5: Accuracy = 0.9110

=== Training Models for Target: cholesterol_ldl_human ===
  Fold 1: Accuracy = 0.6929
  Fold 2: Accuracy = 0.6876
  Fold 3: Accuracy = 0.6830
  Fold 4: Accuracy = 0.6735
  Fold 5: Accuracy = 0.6922

hdl_cholesterol_human Average Accuracy: 0.6821

hemoglobin(hgb)_human Average Accuracy: 0.9097

cholesterol_ldl_human Average Accuracy: 0.6858
